In [1]:
import copy as cp
import numpy as np
import matplotlib as mt
import matplotlib.pyplot as plt
import MDAnalysis as mda
from MDAnalysis.analysis import dihedrals
import pyemma
from tqdm import tqdm

### dihedrals

In [2]:
aw = mda.Universe('pbc_trjs/w.pdb', [f'pbc_trjs/aw{i}.xtc' for i in [1,2,3]])
am = mda.Universe('pbc_trjs/m.pdb', [f'pbc_trjs/am{i}.xtc' for i in [1,2,3]])
bw = mda.Universe('pbc_trjs/w.pdb', 'pbc_trjs/bw.xtc')
bm = mda.Universe('pbc_trjs/m.pdb', [f'pbc_trjs/bm{i}.xtc' for i in [1,2,3]])

/home/msahil/softwares/miniconda3/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [3]:
unis = [aw,bw,am,bm]

In [4]:
names = ['aw','bw','am','bm']

In [20]:
dihedrals = [[] for i in unis]
resids = [[] for i in unis]

for u in range(len(unis)):
    dd = []
    for i in unis[u].residues:
        dd.append(i.phi_selection())
        dd.append(i.psi_selection())
        if i.resname != 'PRO':
            dd.append(i.chi1_selection())
        
    for i in dd:
        try:
            if i == None:
                pass
        except:
            dihedrals[u].append(i)
            resids[u].append([a.resid for a in i.residues])

In [33]:
np.savez('phi_psi_chi1_resids.npz', *resids[0])

In [22]:
len(dihedrals[1])

648

In [40]:
for u in range(len(unis)):
    phi_psi_chi = np.zeros(( int( len(unis[u].trajectory)/2), len(dihedrals[u]) ))
    for i in tqdm( range(phi_psi_chi.shape[0]) ):
        unis[u].trajectory[i*2]
        for j in range(len(dihedrals[u])):
            phi_psi_chi[i,j] = dihedrals[u][j].dihedral.value()
    np.save(f'phi_psi_chi1_{names[u]}.npy', phi_psi_chi)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150001/150001 [1:08:43<00:00, 36.38it/s]


### coordiantes

In [31]:
def get_covar(x,y):
    xm = x-np.mean(x)
    ym = y-np.mean(y)
    return 1/(len(x)-1) * np.sum(xm*ym)

def get_cmat(x):
    ndim=x.shape[1]
    cmt = np.zeros((ndim,ndim))
    for i in range(ndim):
        for j in range(i,ndim):
            cmt[i,j] = cmt[j,i] = get_covar(x[:,i], x[:,j])
    return cmt

def get_inv_sqrt(x):
    el, er = np.linalg.eigh(x)
    return er @ np.diag(1/np.sqrt(el)) @ er.T

def get_whiten(x):
    x = x-np.mean(x)
    for i in range(x.shape[1]):
        x[:,i] = x[:,i]-np.mean(x)
    return get_inv_sqrt( get_cmat(x) ) @ x

In [24]:
positions = np.zeros((200,27))
for a in range(200):
    aw.trajectory[a]
    positions[a] = np.concatenate(( aw.select_atoms('resid 147 to 149 and name N CA C').positions ))

In [25]:
for i in range(positions.shape[1]):
    positions[:,i] = positions[:,i]-np.mean(positions[:,i])

In [27]:
nobs,ndim=positions.shape
cmt = np.zeros((ndim,ndim))
for i in range(ndim):
    for j in range(i,ndim):
        cmt[i,j] = cmt[j,i] = 1/(nobs-1) * np.sum(positions[:,i]*positions[:,j])

In [31]:
el, er = np.linalg.eigh(cmt)

In [34]:
el.shape, er.shape

((27,), (27, 27))

In [39]:
el=np.diag(1/np.sqrt(el))

In [41]:
invsqrt = er @ el @ er.T

In [43]:
invsqrt.shape

(27, 27)

In [44]:
positions

array([[ 0.66955202,  0.41325212,  0.4808495 , ...,  0.36824968,
         0.66494741,  1.04414914],
       [ 0.27954882,  0.90324999,  0.24085164, ...,  0.05825212,
         0.90494909,  0.04415105],
       [ 0.21955126,  0.59324862,  0.25084996, ...,  0.32824877,
         0.52494802,  0.61414884],
       ...,
       [-0.21045286, -0.40675138, -0.44914889, ..., -0.70175001,
         0.09495153, -1.20584895],
       [-0.18045027, -0.42674803, -0.3791492 , ..., -0.24175093,
        -1.01504908, -0.77585055],
       [-0.26044828,  0.14324785,  0.09085011, ..., -0.72175047,
        -0.45505152, -0.80584933]])

In [45]:
positions = positions @ invsqrt

In [7]:
for u in range(len(unis)):
    positions = np.zeros(( int( len(unis[u].trajectory)/2 ), len(unis[u].residues)*3 ))
    for i in tqdm( range(positions.shape[0]) ):
        unis[u].trajectory[i*2]
        positions[i] = np.concatenate(( unis[u].select_atoms('name CA').positions ))
        
    np.save(f'coordinates_ca_{names[u]}.npy', positions)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150001/150001 [07:35<00:00, 329.33it/s]


## reduced dihedrals (20-220)

In [6]:
dih = np.load('phi_psi_chi1_resids.npz')
dih=[dih[i] for i in list(dih)]

In [18]:
dih = [ all( (i>=20) & (i<=220) ) for i in dih ]

In [21]:
np.sum(dih)/len(dih)

0.875

In [30]:
for n in names: np.save(f'reduced_phi_psi_chi1_{n}.npy', np.load(f'phi_psi_chi1_{n}.npy')[:,dih])

## coordinates (mean-free and whitened)

In [56]:
skips=[8,4,8,8]

for a,u in enumerate(unis):
    
    positions = np.zeros(( int(len(u.trajectory)/skips[a]), 2361 ))
    
    for t in tqdm(range(positions.shape[0]), desc=names[a]):
        u.trajectory[t*skips[a]]
        
        positions[t] = np.concatenate((
            u.select_atoms('resid 20 to 147 149 to 220 and name N CA CB C').positions
                                      ))   #getting coordinates
        
    for i in range(positions.shape[1]):   #making mean-free positions
        positions[:,i] -= np.mean(positions[:,i])
        
    nobs, ndim = positions.shape
    cmt = np.zeros((ndim,ndim))  #getting covar matrix
    for i in range(ndim):
        for j in range(i,ndim):
            cmt[i,j] = cmt[j,i] = 1/(nobs-1) * np.sum(positions[:,i]*positions[:,j])
            
    eigval, eigvec = np.linalg.eigh(cmt) # eigen decomposition of covar matrix
    
    #getting inverse square root of cmt 
    # B^2 = cmt^{-1}
    # cmt^{-1/2} = eigvec * eigval^{-1/2} * eigvec^T
    eigval_inv = np.diag(1/np.sqrt(eigval))
    cmt_inv = eigvec @ eigval_inv @ eigvec.T
    
    # getting decorrelated coordinates
    positions = positions @ cmt_inv
    
    np.save(f'coordinates_{names[a]}.npy', positions)

bm: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37500/37500 [00:30<00:00, 1241.38it/s]
